# Modell trainieren, validieren, testen

## <span style="color:red">Schritt 1: Import des Atticus Finch Datensatzes von GitHub</span>

In [1]:
import pandas as pd

# URL des Atticus Finch Datensatzes
url = 'https://raw.githubusercontent.com/OJanz/AtticusFinch/main/AtticusFinchSalesData_v1.csv'  

# Daten in Pandas Data Frame einlesen
df = pd.read_csv(url)

# Liste mit variablen, die bereinigt werden sollen
variablen_bereinigen = ['Frequency', 'Space', 'Sales'] 

# in df nur Variablen behalten, die in der Liste 'variablen_bereinigen' stehen
df = df[variablen_bereinigen] 

df


,Frequency,Space,Sales
0,4740,364,710
1,7703,802,2367
2,9928,926,2325
3,10139,1269,3284
4,11170,1149,2429
...,...,...,...
154,84673,3426,16850
155,87728,3772,27607
156,88889,3772,27607
157,90824,3590,9387


## <span style="color:red">Schritt 2: Ausreißer eliminieren</span>

In [2]:
#--------------------------------------------------------
# FUNKTION ZUR ELIMINIERUNG VON AUSREISSERN
#--------------------------------------------------------

def remove_outliers_iqr(df):
    
    # IQR berechnen und die Grenzen für Ausreißer je Variable (Spalte) berechnen
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Daten filtern, um Ausreißer zu entfernen
    outliers = df[((df < lower_bound) | (df > upper_bound)).any(axis=1)]
    
    # DataFrame mit den Ausreißern erstellen
    no_outliers = df[~df.index.isin(outliers.index)]

    # bereinigte Daten und Ausreißer zurückgeben
    return no_outliers, outliers

In [3]:
df_no_outliers, df_outliers = remove_outliers_iqr(df)

df = df_no_outliers

print(f'Anzahl eliminierter Stores: {len(df_outliers)}')
print(f'Anzahl Stores nach Bereinigung: {len(df_no_outliers)}')

Anzahl eliminierter Stores: 21
Anzahl Stores nach Bereinigung: 138


## <span style="color:red">Schritt 3: Train-, Validate-, Test-Split</span>

In [4]:
from sklearn.model_selection import train_test_split

# Prädiktoren und abhängige Variable X bzw. y zuordnen
X = df[['Frequency', 'Space']]
y = df['Sales']

# Daten in Train-, Validation- und Test-Set splitten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

print(f'Umfang Train-Set: {len(X_train)}')
print(f'Umfang Validation-Set: {len(X_val)}')
print(f'Umfang Test-Set: {len(X_test)}')

Umfang Train-Set: 82
Umfang Validation-Set: 28
Umfang Test-Set: 28


## <span style="color:red">Schritt 4: Scaling der Prädiktoren</span>

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Feature-Scaling durchführen (Prädiktoren standardisieren)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Scaler auf X_train anpassen

# Auf X_train berechneten Scaler auf X_val und X_test anwenden
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print(f'Mittelwerte der Prädiktoren in X_train: {np.mean(X_train, axis=0).round(2)}')
print(f'Standardabweichung der Prädiktoren in X_train: {np.std(X_train, axis=0).round(2)}')

print(f'\nMittelwerte der Prädiktoren in X_val: {np.mean(X_val, axis=0).round(2)}')
print(f'Standardabweichung der Prädiktoren in X_val: {np.std(X_val, axis=0).round(2)}')

print(f'\nMittelwerte der Prädiktoren in X_test: {np.mean(X_val, axis=0).round(2)}')
print(f'Standardabweichung der Prädiktoren in X_test: {np.std(X_val, axis=0).round(2)}')

Mittelwerte der Prädiktoren in X_train: [ 0. -0.]
Standardabweichung der Prädiktoren in X_train: [1. 1.]

Mittelwerte der Prädiktoren in X_val: [ 0.2  -0.04]
Standardabweichung der Prädiktoren in X_val: [0.99 0.92]

Mittelwerte der Prädiktoren in X_test: [ 0.2  -0.04]
Standardabweichung der Prädiktoren in X_test: [0.99 0.92]


## <span style="color:red">Schritt 5: Modelle trainieren und validieren</span>

In [7]:
# Verschiedene lineare Regressionsmodelle importieren
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Funktionen zur Berechnung der Gütemaße importieren
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [8]:
# LINEAR REGRESSION
# ---------------------------

# Modell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Validiere das Modell
y_val_pred = model.predict(X_val)
print('Validation MAE:', mean_absolute_error(y_val, y_val_pred).round(4))
print('Validation RMSE:', np.sqrt(mean_squared_error(y_val, y_val_pred)).round(4))

# Berechnung von Adjusted R2
r2 = r2_score(y_val, y_val_pred)
n = X_val.shape[0]  # Anzahl der Beobachtungen
p = X_val.shape[1]  # Anzahl der Prädiktoren
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print('Validation Adjusted R2:', adjusted_r2.round(4))

Validation MAE: 863.6205
Validation RMSE: 1039.7586
Validation Adjusted R2: 0.7647


In [9]:
# RANDOM FORREST
# ---------------------------

# Modell trainieren
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Validiere das Modell
y_val_pred = model.predict(X_val)
print('Validation MAE:', mean_absolute_error(y_val, y_val_pred).round(4))
print('Validation RMSE:', np.sqrt(mean_squared_error(y_val, y_val_pred)).round(4))

# Berechnung von Adjusted R2
r2 = r2_score(y_val, y_val_pred)
n = X_val.shape[0]  # Anzahl der Beobachtungen
p = X_val.shape[1]  # Anzahl der Prädiktoren
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print('Validation Adjusted R2:', adjusted_r2.round(4))

Validation MAE: 682.1264
Validation RMSE: 785.1711
Validation Adjusted R2: 0.8658


In [10]:
# GRADIENT BOOSTING
# ---------------------------

# Modell trainieren
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Validiere das Modell
y_val_pred = model.predict(X_val)
print('Validation MAE:', mean_absolute_error(y_val, y_val_pred).round(4))
print('Validation RMSE:', np.sqrt(mean_squared_error(y_val, y_val_pred)).round(4))

# Berechnung von Adjusted R2
r2 = r2_score(y_val, y_val_pred)
n = X_val.shape[0]  # Anzahl der Beobachtungen
p = X_val.shape[1]  # Anzahl der Prädiktoren
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print('Validation Adjusted R2:', adjusted_r2.round(4))

Validation MAE: 586.3782
Validation RMSE: 742.4168
Validation Adjusted R2: 0.88


## <span style="color:red">Schritt 6: Modell testen</span>

In [11]:
# Teste das Modell
y_test_pred = model.predict(X_test)
print('Test MAE:', mean_absolute_error(y_test, y_test_pred).round(4))
print('Test RMSE:', np.sqrt(mean_squared_error(y_test, y_test_pred)).round(4))

# Berechnung von Adjusted R2
r2 = r2_score(y_test, y_test_pred)
n = X_val.shape[0]  # Anzahl der Beobachtungen
p = X_val.shape[1]  # Anzahl der Prädiktoren
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print('Validation Adjusted R2:', adjusted_r2.round(4))

Test MAE: 574.77
Test RMSE: 727.0143
Validation Adjusted R2: 0.8439


## <span style="color:red">Schritt 7: Umsatzprognose für neue Stores</span>

In [12]:
# Vorhersage für neue Stores
new_Stores = pd.DataFrame([[45000, 1500], [40000, 1700]], columns=['Frequency', 'Space'])  

# Auf X_train berechneten Scaler auf new_Stores anwenden
new_Stores = scaler.transform(new_Stores)  # Feature Scaling

vorhersage = model.predict(new_Stores)
print('Vorhersage für neue Werte:', vorhersage.round(0))


Vorhersage für neue Werte: [6018. 4591.]
